In [68]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
pd.options.display.float_format = '{:,.2f}'.format

In [69]:
# 2017 Reform
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\2017_law.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
reform_old = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})

pol_old = tc.Policy()
pol_old.implement_reform(reform_old)

#Baseline Calculator
calc_old=tc.Calculator(records = tc.Records(), policy = pol_old)
calc_old.advance_to_year(2018)
calc_old.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [70]:
calc_new = tc.Calculator(records = tc.Records(), policy = tc.Policy())
calc_new.advance_to_year(2018)
calc_new.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [71]:
vlist = ['c00100', 'iitax', 'standard', 'c04470', 'c19200', 'c19700', 'n24', 'c07220', 'c11070', 's006']

In [72]:
odf_full = calc_old.dataframe(vlist)
ndf_full = calc_new.dataframe(vlist)

In [73]:
def getstat(df, title, var = 's006'):
    print(title + ": " + "{:,.0f}".format((df[var].sum())))

In [74]:
def w_avg(df, var):
    return (df[var] * df['s006']).sum() / df['s006'].sum()

# Section 1: Number of Filers

## 1a. Number of Tax Filers with Tax Liability > \$0

In [75]:
getstat(odf_full[odf_full['iitax'] > 0], 'Pre-TCJA')
getstat(ndf_full[ndf_full['iitax'] > 0], 'TCJA')

Pre-TCJA: 103,354,373
TCJA: 97,069,932


## 1b. Number of Tax Filers Claiming Standard, Itemized Deduction

In [76]:
for df in [(odf_full, 'Pre-TCJA'), (ndf_full, 'TCJA')]:
    getstat(df[0][(df[0]['iitax'] > 0) & (df[0]['standard'] > 0)], df[1] + ', Standard Deduction, Positive Income Tax Liability')
    getstat(df[0][(df[0]['iitax'] > 0) & (df[0]['c04470'] > 0)], df[1] + ', Itemized Deduction, Positive Income Tax Liability')
    getstat(df[0][df[0]['standard'] > 0], df[1] + ', Standard Deduction, All Tax Filers')
    getstat(df[0][df[0]['c04470'] > 0], df[1] + ', Itemized Deduction, All Tax Filers')

Pre-TCJA, Standard Deduction, Positive Income Tax Liability: 60,869,564
Pre-TCJA, Itemized Deduction, Positive Income Tax Liability: 42,466,657
Pre-TCJA, Standard Deduction, All Tax Filers: 127,618,535
Pre-TCJA, Itemized Deduction, All Tax Filers: 45,164,447
TCJA, Standard Deduction, Positive Income Tax Liability: 78,528,593
TCJA, Itemized Deduction, Positive Income Tax Liability: 18,517,343
TCJA, Standard Deduction, All Tax Filers: 153,399,973
TCJA, Itemized Deduction, All Tax Filers: 19,396,400


In [77]:
odf_posagi = copy.deepcopy(odf_full[odf_full['c00100'] > 0])
ndf_posagi = copy.deepcopy(ndf_full[ndf_full['c00100'] > 0])

## 1c. Number Claiming Standard, Itemized Deductions by AGI Decile

In [78]:
numbin = 10
tc.add_quantile_bins(odf_posagi, income_measure = 'c00100', num_bins = numbin)
tc.add_quantile_bins(ndf_posagi, income_measure = 'c00100', num_bins = numbin)
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Standard Deduction', [])
        decile_dict[df[1] + ', Standard Deduction'].append(bindf[bindf['standard'] > 0]['s006'].sum())
        decile_dict.setdefault(df[1] + ', Itemized Deduction', [])
        decile_dict[df[1] + ', Itemized Deduction'].append(bindf[bindf['c04470'] > 0]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)

decile_df.index = list(range(1,11)) + ['Total']
decile_df

,"Pre-TCJA, Itemized Deduction","Pre-TCJA, Standard Deduction","TCJA, Itemized Deduction","TCJA, Standard Deduction"
1,"3,814.55","16,095,339.48","19,641.76","16,095,306.26"
2,"21,155.82","16,093,592.51","13,961.97","16,102,539.71"
3,"532,534.22","15,587,796.42","155,843.65","15,966,403.39"
4,"1,075,383.14","15,044,814.98","371,552.69","15,746,564.44"
5,"2,007,518.29","14,111,940.89","608,521.93","15,509,551.40"
6,"3,373,580.96","12,745,360.58","1,008,938.03","15,110,474.73"
7,"5,356,757.03","10,756,904.12","1,867,818.57","14,248,962.30"
8,"7,699,349.45","8,419,487.16","2,928,573.30","13,190,149.43"
9,"10,814,196.70","5,306,380.43","4,163,794.59","11,958,004.78"
10,"14,280,078.72","1,839,691.77","8,257,742.70","7,862,109.64"


## 1d. Number Claiming Interest Deduction

In [79]:
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Interest Deduction', [])
        decile_dict[df[1] + ', Interest Deduction'].append(bindf[(bindf['c19200'] > 0) & (bindf['c04470'] > 0)]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)
decile_df.index  = list(range(1,11)) + ['Total']
decile_df

,"Pre-TCJA, Interest Deduction","TCJA, Interest Deduction"
1,0.00,"4,345.93"
2,"13,339.02","10,140.00"
3,"290,814.17","90,823.67"
4,"611,190.42","204,894.94"
5,"1,256,671.79","396,932.69"
6,"2,293,844.22","731,014.40"
7,"4,001,049.69","1,531,990.05"
8,"6,115,719.36","2,475,697.12"
9,"9,133,761.45","3,678,619.78"
10,"12,170,547.80","7,550,666.81"


## 1e. Number Claiming Charitable Deduction

In [80]:
decile_dict = {}
for df in [(odf_posagi, 'Pre-TCJA'), (ndf_posagi, 'TCJA')]:
    for i in range(1, numbin + 1):
        bindf = df[0][df[0]['bins'] == i]
        decile_dict.setdefault(df[1] + ', Charity Deduction', [])
        decile_dict[df[1] + ', Charity Deduction'].append(bindf[(bindf['c19700'] > 0) & (bindf['c04470'] > 0)]['s006'].sum())
decile_df = pd.DataFrame(decile_dict)
decile_df = decile_df.append(decile_df.sum(numeric_only=True), ignore_index=True)
decile_df.index  = list(range(1,11)) + ['Total']
decile_df

,"Pre-TCJA, Charity Deduction","TCJA, Charity Deduction"
1,0.00,"7,026.73"
2,"10,754.33","6,770.70"
3,"321,271.86","85,225.67"
4,"685,968.27","214,625.86"
5,"1,430,103.42","450,599.88"
6,"2,457,727.73","789,208.82"
7,"4,082,518.79","1,458,130.42"
8,"6,309,127.95","2,491,392.71"
9,"9,259,098.23","3,676,412.75"
10,"13,086,027.57","7,755,702.15"


# Section 2: Marginal Tax Rates on Labor

## 2a. Effective Marginal Tax Rate on Labor

In [81]:
odf_full['mtr'] = calc_old.mtr()[1]
ndf_full['mtr'] = calc_new.mtr()[1]

In [82]:
print('Pre-TCJA, Average EMTR: ' + str(w_avg(odf_full, 'mtr')))
print('TCJA, Average EMTR: ' + str(w_avg(ndf_full, 'mtr')))

Pre-TCJA, Average EMTR: 0.11580142502096552
TCJA, Average EMTR: 0.09189758044937907


# Section 3: The Child Tax Credit

## 3a./3b. Number of Taxpayers Claiming CTC (Pre-TCJA)

In [83]:
getstat(odf_full[(odf_full['iitax'] > 0) & ((odf_full['c07220'] > 0) | (odf_full['c11070'] > 0))], 'Pre-TCJA: Positive Tax Liability')
getstat(odf_full[(odf_full['iitax'] <= 0) & ((odf_full['c07220'] > 0) | (odf_full['c11070'] > 0))], 'Pre-TCJA: 0/Negative Tax Liability')
getstat(odf_full[(odf_full['c07220'] > 0) | (odf_full['c11070'] > 0)], 'Pre-TCJA: All Tax Filers')

Pre-TCJA: Positive Tax Liability: 14,103,647
Pre-TCJA: 0/Negative Tax Liability: 20,169,227
Pre-TCJA: All Tax Filers: 34,272,874


## 3c./3d. Number of Taxpayers Claiming CTC (TCJA)

In [84]:
getstat(ndf_full[(ndf_full['iitax'] > 0) & ((ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0))], 'TCJA: Positive Tax Liability')
getstat(ndf_full[(ndf_full['iitax'] <= 0) & ((ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0))], 'TCJA: 0/Negative Tax Liability')
getstat(ndf_full[(ndf_full['c07220'] > 0) | (ndf_full['c11070'] > 0)], 'TCJA: All Tax Filers')

TCJA: Positive Tax Liability: 18,016,594
TCJA: 0/Negative Tax Liability: 22,485,246
TCJA: All Tax Filers: 40,501,840


## 3e. Average Tax Burden by Family Size, AGI < \$250,000

In [85]:
ch_dict = {}
for df in [(odf_full[(odf_full['c00100'] < 250000)], 'Pre-TCJA'), (ndf_full[(ndf_full['c00100'] < 250000)], 'TCJA')]:
    for i in range(0, 4):
        bindf = df[0][df[0]['n24'] == i]
        taxsum = (bindf['iitax'] * bindf['s006']).sum()
        incsum = (bindf['c00100'] * bindf['s006']).sum()
        ch_dict.setdefault(df[1] + ', Average Income Tax', [])
        ch_dict[df[1] + ', Average Income Tax'].append(taxsum / bindf['s006'].sum())
        ch_dict.setdefault(df[1] + ', Average Tax Rate', [])
        ch_dict[df[1] + ', Average Tax Rate'].append(taxsum / incsum)
decile_df = pd.DataFrame(ch_dict)
decile_df

,"Pre-TCJA, Average Income Tax","Pre-TCJA, Average Tax Rate","TCJA, Average Income Tax","TCJA, Average Tax Rate"
0,"4,621.28",0.11,"4,071.30",0.09
1,"3,406.54",0.06,"2,258.38",0.04
2,"2,858.19",0.04,"1,297.19",0.02
3,"1,411.65",0.02,-397.14,-0.01


# Section 4: Charitable Giving

## 4a. Number of Taxpayers Claiming Charitable Deduction

In [86]:
getstat(odf_full[(odf_full['c19700'] > 0) & (odf_full['c04470'] > 0)], 'Pre-TCJA, All Filers, Number Taking Charitable Deduction')
getstat(odf_full[(odf_full['iitax'] > 0) & (odf_full['c19700'] > 0) & (odf_full['c04470'] > 0)], 'Pre-TCJA, Positive Tax Liability, Number Taking Medical Deduction')

getstat(ndf_full[(ndf_full['c19700'] > 0) & (ndf_full['c04470'] > 0)], 'TCJA, All Filers, Number Taking Charitable Deduction')
getstat(ndf_full[(ndf_full['iitax'] > 0) & (ndf_full['c19700'] > 0) & (ndf_full['c04470'] > 0)], 'TCJA, Positive Tax Liability, Number Taking Medical Deduction')


Pre-TCJA, All Filers, Number Taking Charitable Deduction: 37,642,598
Pre-TCJA, Positive Tax Liability, Number Taking Medical Deduction: 35,796,801
TCJA, All Filers, Number Taking Charitable Deduction: 16,935,096
TCJA, Positive Tax Liability, Number Taking Medical Deduction: 16,358,026
